<a href="https://colab.research.google.com/github/Dipak22/ReinforcementLearning/blob/master/LunarLander_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objectives

1. Able to use gynmasium , the environment library
2. Able to use stable-baselines3, the deep reinforcement learning library.
3. Able to push our model to huggingface hub




In [1]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (727 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 125044 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [2]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

INFO: pip is looking at multiple versions of gymnasium[box2d] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━

### *In order to generate a replay video,  we need a virtual screen to be able to render the environment(and thus record frames)*

In [3]:
!sudo apt-get update
!sudo apt-get install -y python2-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,378 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,757 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://ppa.launchpadc

### To make sure the installed librraries are used, sometimes it is required to restart the notebook runtime

In [ ]:
import os
os.kill(os.getpid(),9)

In [2]:
#virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible = 0, size=(1400,900))
virtual_display.start()

In [3]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

### Gymnasium library provides
1. An interface to create RL environments.
2. A collection of environments.

-- to create environment `gymnasium.make()`
-- to reset the environment to its initial state `observation = env.reset()`

-- `env.step()` returns
. `observation` : The new state(s1)
`reward` : The reward we get after executing the action.
`terminated`: Indicates if the episode terminated.
`truncated`: it indicates a timelimit or if an agent go out of bounds of the environment for instance.
`info`: A dictionary that provides additional info.


In [5]:
import gymnasium as gym

#First create our env called LunarLander-v2
env = gym.make("LunarLander-v2")

# rest the environmnet to its initial state
observation, info = env.reset()

for _ in range(20):
  #pick a random action
  action = env.action_space.sample()
  print(f"Action taken : {action}")

  # take the action
  observation, reward, terminated, truncated, info = env.step(action)
  if terminated or truncated:
    # Reset the environment
    print("Environment is rest")
    observation, info = env.reset()

env.close()


Action taken : 0
Action taken : 0
Action taken : 0
Action taken : 2
Action taken : 2
Action taken : 3
Action taken : 2
Action taken : 3
Action taken : 3
Action taken : 0
Action taken : 1
Action taken : 1
Action taken : 2
Action taken : 0
Action taken : 2
Action taken : 3
Action taken : 0
Action taken : 1
Action taken : 3
Action taken : 2


In [7]:
# Observe the environment
env = gym.make("LunarLander-v2")
env.reset()
print(f"Observation space {env.observation_space.shape}")
print(f"Sample observation {env.observation_space.sample()}")

Observation space (8,)
Sample observation [-85.24212     61.583855    -4.6979375    1.3172936   -0.62413055
  -0.22470331   0.9644566    0.79328793]


In [10]:
# observe action space
print(f"Action space shape {env.action_space.n}")
print(f"Action space sample: {env.action_space.sample()}")

Action space shape 4
Action space sample: 2


In [9]:
env.action_space

Discrete(4)

The action space (the set of possible actions the agent can take) is discrete with 4 actions available 🎮:

Action 0: Do nothing,
Action 1: Fire left orientation engine,
Action 2: Fire the main engine,
Action 3: Fire right orientation engine.
Reward function (the function that will give a reward at each timestep) 💰:

After every step a reward is granted. The total reward of an episode is the sum of the rewards for all the steps within that episode.

For each step, the reward:

Is increased/decreased the closer/further the lander is to the landing pad.
Is increased/decreased the slower/faster the lander is moving.
Is decreased the more the lander is tilted (angle not horizontal).
Is increased by 10 points for each leg that is in contact with the ground.
Is decreased by 0.03 points each frame a side engine is firing.
Is decreased by 0.3 points each frame the main engine is firing.
The episode receive an additional reward of -100 or +100 points for crashing or landing safely respectively.

An episode is considered a solution if it scores at least 200 points.

### Vectorized Environment
> We create a vectorized env ( stacking multiple envs into a single env) of 16 envs

In [11]:
env = make_vec_env("LunarLander-v2", n_envs = 16)

In [12]:
model  =PPO(policy = 'MlpPolicy',
            env = env,
            n_steps = 1024,
            batch_size = 64,
            n_epochs = 4,
            gamma = 0.999,
            gae_lambda = 0.98,
            ent_coef = 0.01,
            verbose = 1)

Using cuda device


In [13]:
# Train it for 1,000,000 timesteps
model.learn(total_timesteps=1000000)
# Save the model
model_name = "ppo-LunarLander-v2"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 91       |
|    ep_rew_mean     | -171     |
| time/              |          |
|    fps             | 2667     |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 92.5         |
|    ep_rew_mean          | -141         |
| time/                   |              |
|    fps                  | 2085         |
|    iterations           | 2            |
|    time_elapsed         | 15           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.008652566  |
|    clip_fraction        | 0.0406       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | 0.0024622679 |
|    learning_r

In [14]:
#Evaluate the model/agent
eval_env = Monitor(gym.make("LunarLander-v2", render_mode = "rgb_array"))
mean_reward, std_reward  = evaluate_policy(model,
                                           eval_env,
                                           n_eval_episodes = 10,
                                           deterministic = True)
print(f"mean_reward: {mean_reward:.2f} +/- {std_reward}")

mean_reward: 262.87 +/- 21.020867958333053


In [15]:
#Publish the model to huggingface hub
notebook_login()

Token has not been saved to git credential helper.


### Upload the model

In [16]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

env_id = "LunarLander-v2"

model_architecture = "PPO"
repo_id = "dasdipak/ppo-LunarLander-v2"
commit_message = "Upload trained PPO LunarLander-v2 agent"

eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode = "rgb_array")])

package_to_hub(model = model,
               model_name = model_name,
               model_architecture = model_architecture,
               env_id = env_id,
               eval_env = eval_env,
               repo_id = repo_id,
               commit_message = commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmppczj01_r/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmppczj01_r/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmppczj01_r/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmppczj01_r/-step-0-to-step-1000.mp4
ℹ Pushing repo dasdipak/ppo-LunarLander-v2 to the Hugging Face Hub


ppo-LunarLander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

policy.optimizer.pth:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

replay.mp4:   0%|          | 0.00/161k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/dasdipak/ppo-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/dasdipak/ppo-LunarLander-v2/commit/6dca4d7ac0a6a3e99404927151651520e2f176b0', commit_message='Upload trained PPO LunarLander-v2 agent', commit_description='', oid='6dca4d7ac0a6a3e99404927151651520e2f176b0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dasdipak/ppo-LunarLander-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='dasdipak/ppo-LunarLander-v2'), pr_revision=None, pr_num=None)

### Loading the model from the hub.


In [17]:
from huggingface_sb3 import load_from_hub
repo_id = "dasdipak/ppo-LunarLander-v2"
filename = "ppo-LunarLander-v2.zip"

checkpoint = load_from_hub(repo_id, filename)
model = PPO.load(checkpoint, print_system_info=True)

ppo-LunarLander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

== CURRENT SYSTEM INFO ==
- OS: Linux-6.1.85+-x86_64-with-glibc2.35 # 1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
- Python: 3.11.11
- Stable-Baselines3: 2.0.0a5
- PyTorch: 2.6.0+cu124
- GPU Enabled: True
- Numpy: 2.0.2
- Cloudpickle: 3.1.1
- Gymnasium: 0.28.1
- OpenAI Gym: 0.25.2

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-6.1.85+-x86_64-with-glibc2.35 # 1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
- Python: 3.11.11
- Stable-Baselines3: 2.0.0a5
- PyTorch: 2.6.0+cu124
- GPU Enabled: True
- Numpy: 2.0.2
- Cloudpickle: 3.1.1
- Gymnasium: 0.28.1
- OpenAI Gym: 0.25.2



In [18]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=273.07 +/- 16.66660795236586
